In [70]:
import json
from dotenv import load_dotenv
import os
import base64
from requests import post

In [71]:
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [72]:
import requests
import base64

def get_access_token(client_id, client_secret):
    auth_url =  "https://accounts.spotify.com/api/token"
    auth_header ={
        "Authorization": f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    auth_date = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    auth_response = requests.post(auth_url, data=auth_date, headers=auth_header)
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    return access_token

In [73]:
def get_genres(access_token, limit=50, offset=0):
    genres_url = 'https://api.spotify.com/v1/browse/categories'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'limit': limit,
        'offset': offset
    }
    response = requests.get(genres_url, headers=headers, params=params)
    genres_data = response.json()
    genres = [(category['id'], category['name']) for category in genres_data['categories']['items']]
    return genres

In [74]:
print('List of Generes:')
token = get_access_token(client_id, client_secret)
genres = get_genres(token)
for genre_id, genre_name in genres:
    print(f"{genre_name}")

List of Generes:
New Releases
Hip-Hop
Pop
Country
Latin
Charts
Student
Rock
Dance/Electronic
In the car
Discover
Mood
Indie
Disney
R&B
Christian & Gospel
Workout
Música Mexicana
La Tierra del Corrido
K-pop
Chill
Netflix
Sleep
Party
At Home
Decades
Love
Metal
Jazz
Trending
Frequency
Classical
Folk & Acoustic
Focus
Soul
Kids & Family
Gaming
Anime
TV & Movies
Instrumental
Punk
Ambient
Blues
Cooking & Dining
Alternative
Travel
Caribbean
Afro
Songwriters
Nature & Noise


In [75]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Load environment variables
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Set up Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_artist_genre(artist_id):
    artist = sp.artist(artist_id)
    return artist['genres']


In [76]:
import pandas as pd
import time

def create_dataframe(query_list, limit=50):
    data = {
        'Track_ID': [],
        'Track_name': [],
        'Artist_name': [],
        'Popularity_score': [],
        'Release_year': [],
        'Genre': []
    }

    for query in query_list:
        search_results = sp.search(query, limit=limit, type='track')
        tracks = search_results['tracks']['items']

        for track in tracks:
            data['Track_ID'].append(track['id'])
            data['Track_name'].append(track['name'])
            data['Artist_name'].append(track['artists'][0]['name'])
            data['Popularity_score'].append(track['popularity'])
            data['Release_year'].append(track['album']['release_date'][:4])
            data['Genre'].append(query)

        time.sleep(2)

    df = pd.DataFrame(data)
    return df

token = get_access_token(client_id, client_secret)
genres = get_genres(token)

def get_genres_list(token):
    genres = get_genres(token)  # Retrieve genres using the token

    # Format genres with quotes and "genre:" prefix:
    genres_with_quote = ['"genre:{}"'.format(genre) for genre in genres]
    return genres_with_quote

genres_list = get_genres_list(token)

df = create_dataframe(genres_list, limit=50)

In [78]:
df['Genre'].value_counts()

Genre
"genre:('0JQ5DAqbMKFQ00XGBls6ym', 'Hip-Hop')"                  50
"genre:('0JQ5DAqbMKFxXaXKP7zcDp', 'Latin')"                    50
"genre:('0JQ5DAqbMKFPrEiAOxgac3', 'Classical')"                50
"genre:('0JQ5DAqbMKFy78wprEpAjl', 'Folk & Acoustic')"          50
"genre:('0JQ5DAqbMKFCbimwdOYlsl', 'Focus')"                    50
"genre:('0JQ5DAqbMKFIpEuaCnimBj', 'Soul')"                     50
"genre:('0JQ5DAqbMKFFoimhOqWzLB', 'Kids & Family')"            50
"genre:('0JQ5DAqbMKFCfObibaOZbv', 'Gaming')"                   50
"genre:('0JQ5DAqbMKFziKOShCi009', 'Anime')"                    50
"genre:('0JQ5DAqbMKFAjfauKLOZiv', 'Punk')"                     50
"genre:('0JQ5DAqbMKFLjmiZRss79w', 'Ambient')"                  50
"genre:('0JQ5DAqbMKFQiK2EHwyjcU', 'Blues')"                    50
"genre:('0JQ5DAqbMKFRY5ok2pxXJ0', 'Cooking & Dining')"         50
"genre:('0JQ5DAqbMKFFtlLYUHv8bT', 'Alternative')"              50
"genre:('0JQ5DAqbMKFAQy4HL4XU2D', 'Travel')"                   50
"gen

In [79]:
#Function to retrive each tracks audio features

def get_audio_features(track_ids):
    features_list = []

    for i in range(0, len(track_ids), 50):
        batch = track_ids[i:i + 50]
        feature_results = sp.audio_features(batch)
        features_list += feature_results
    
    return features_list

In [80]:
track_ids = df['Track_ID'].tolist()
audio_features = get_audio_features(track_ids)

In [84]:
audio_features

[{'danceability': 0.862,
  'energy': 0.642,
  'key': 6,
  'loudness': -13.652,
  'mode': 0,
  'speechiness': 0.101,
  'acousticness': 0.102,
  'instrumentalness': 0,
  'liveness': 0.272,
  'valence': 0.765,
  'tempo': 99.2,
  'type': 'audio_features',
  'id': '1SyQ6t9RdRBK0QUCS6a797',
  'uri': 'spotify:track:1SyQ6t9RdRBK0QUCS6a797',
  'track_href': 'https://api.spotify.com/v1/tracks/1SyQ6t9RdRBK0QUCS6a797',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1SyQ6t9RdRBK0QUCS6a797',
  'duration_ms': 267267,
  'time_signature': 4},
 {'danceability': 0.373,
  'energy': 0.967,
  'key': 1,
  'loudness': -1.36,
  'mode': 1,
  'speechiness': 0.202,
  'acousticness': 0.253,
  'instrumentalness': 7.27e-05,
  'liveness': 0.499,
  'valence': 0.666,
  'tempo': 139.03,
  'type': 'audio_features',
  'id': '2dvgoGTEjoPWou8KdL9h1g',
  'uri': 'spotify:track:2dvgoGTEjoPWou8KdL9h1g',
  'track_href': 'https://api.spotify.com/v1/tracks/2dvgoGTEjoPWou8KdL9h1g',
  'analysis_url': 'https://api.spoti

In [86]:
import pandas as pd

# Assuming audio_features is a list containing dictionaries as elements
filtered_audio_features = [feature for feature in audio_features if feature is not None]

# Create DataFrame
audio_features_df = pd.DataFrame(filtered_audio_features)

In [89]:
merged_df = df.merge(audio_features_df, left_on='Track_ID', right_on='id', how='inner')

In [92]:
unique_df = merged_df.drop_duplicates(subset='Track_ID', keep='first')

In [96]:
unique_df.columns

Index(['Track_ID', 'Track_name', 'Artist_name', 'Popularity_score',
       'Release_year', 'Genre', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature'],
      dtype='object')

In [95]:
unique_df.head()

,Track_ID,Track_name,Artist_name,Popularity_score,Release_year,Genre,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1SyQ6t9RdRBK0QUCS6a797,Hip Hop Hooray,Naughty By Nature,69,1993,"""genre:('0JQ5DAqbMKFQ00XGBls6ym', 'Hip-Hop')""",0.862,0.642,6,-13.652,...,0.2720,0.765,99.200,audio_features,1SyQ6t9RdRBK0QUCS6a797,spotify:track:1SyQ6t9RdRBK0QUCS6a797,https://api.spotify.com/v1/tracks/1SyQ6t9RdRBK...,https://api.spotify.com/v1/audio-analysis/1SyQ...,267267,4
1,2dvgoGTEjoPWou8KdL9h1g,WHAT KIND OF RAPPIN' IS THIS?,JPEGMAFIA,58,2021,"""genre:('0JQ5DAqbMKFQ00XGBls6ym', 'Hip-Hop')""",0.373,0.967,1,-1.360,...,0.4990,0.666,139.030,audio_features,2dvgoGTEjoPWou8KdL9h1g,spotify:track:2dvgoGTEjoPWou8KdL9h1g,https://api.spotify.com/v1/tracks/2dvgoGTEjoPW...,https://api.spotify.com/v1/audio-analysis/2dvg...,112452,4
2,32af3BQBta1DxB96RtdKzB,Hip Hop Hippies,The Dining Rooms,42,1999,"""genre:('0JQ5DAqbMKFQ00XGBls6ym', 'Hip-Hop')""",0.697,0.478,9,-13.435,...,0.1090,0.175,159.957,audio_features,32af3BQBta1DxB96RtdKzB,spotify:track:32af3BQBta1DxB96RtdKzB,https://api.spotify.com/v1/tracks/32af3BQBta1D...,https://api.spotify.com/v1/audio-analysis/32af...,184921,4
3,3xI2BX87nRys1ENw1Zl0El,Hip Hop,DJ Khaled,39,2012,"""genre:('0JQ5DAqbMKFQ00XGBls6ym', 'Hip-Hop')""",0.646,0.859,11,-4.934,...,0.1080,0.628,77.057,audio_features,3xI2BX87nRys1ENw1Zl0El,spotify:track:3xI2BX87nRys1ENw1Zl0El,https://api.spotify.com/v1/tracks/3xI2BX87nRys...,https://api.spotify.com/v1/audio-analysis/3xI2...,263213,4
4,6R5WK01wmNhiZfjjnErIyO,Hip Hop,Otesla,36,2018,"""genre:('0JQ5DAqbMKFQ00XGBls6ym', 'Hip-Hop')""",0.790,0.281,6,-8.897,...,0.0883,0.837,98.117,audio_features,6R5WK01wmNhiZfjjnErIyO,spotify:track:6R5WK01wmNhiZfjjnErIyO,https://api.spotify.com/v1/tracks/6R5WK01wmNhi...,https://api.spotify.com/v1/audio-analysis/6R5W...,107518,4
